<a href="https://colab.research.google.com/github/yuvalchemke/Image-process/blob/main/Web_app_Yuval_Chemke.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Hello to you all!**

Welcome to my web-app, in this page you can do some image processing and analysis.

Don't forget to smile!(:

**Instructions**

Follow the windows according to their order

Click on the play button that appears on the left side of each window

In [ ]:
#@title 1. Before starting, run this cell please and mount here your drive. { display-mode: "form" }

# mount Google Drive
from google.colab import drive
drive.mount('/content/drive')             

# Liberies
import numpy as np      # mathematical functions, random number generators, linear algebra routines, Fourier transforms, and more.
import skimage
import skimage.io as io # io is an input output moduile, as part of skimage
from skimage import img_as_float # Convert an image to floating point format, with values in [0, 1]
from skimage import filters      #functions transforming images into other images 
from skimage import color
from skimage import io
import glob             # The glob module finds all the path names
                        # The glob.glob returns the list of files with their full path 

import matplotlib.pyplot as plt #importing pyplot to display our image !important! RGB
import cv2              # openCV,   !important! BGR
from google.colab.patches import cv2_imshow   #Google colab crashes if you try to display image using cv2.imshow() instead import from google.colab.patches import cv2_imshow and display
import os               # creating and removing a directory (folder), fetching its contents, changing etc.

import requests # lib to request image from web
import shutil # lib to save the image locally
import imutils

import scipy.ndimage as ndi  #ndimage (n-dimensional image) package, that contains the number of general image processing and analysis functions

import ipywidgets as widgets
from IPython.display import display


# Defining a working folder, using a variable
folder_path = '/content'




In [ ]:
#@title 2. Upload an image from your computer. { display-mode: "form" }
# add code here to upload images, using google.colab.files.upload()
from google.colab import files

try:
  uploaded = files.upload()
except:
  print("")
  print("Please use Chrome, and enable cookies!")
  print("cookie אנא היכנסו דרך דפדפן כרום במחשב, והפעילו גישה לקבצי ")
  # lets the user upload the file, and stores the file name in 'file_names'. The files are uploaded under /content/FILE_NAME
file_names = uploaded.keys()
# lets convert our file names into a list of of file names
list_file_paths = list(file_names)
img = io.imread(list_file_paths[0])

In [ ]:
#@title 3. Is this the photo you want? { display-mode: "form" }

plt.figure(figsize=(5,5))                                              # the size of the image on the screen
plt.imshow(img)                                                        # display image 

button1 = widgets.Button(description="Yes (:")
output1 = widgets.Output()

button2 = widgets.Button(description="NO  ):")
output2 = widgets.Output()

def on_button_clicked(b):
  # Display the message within the output widget.
  if b == button1:
    with output1:
      print("Great! continue to the next step.")
  else:
    with output1:
      print("Oh NO!!! let's try again, pls go one step backwards and press the play button")

button1.on_click(on_button_clicked)
display(button1, output1)

button2.on_click(on_button_clicked)
display(button2, output2)

In [ ]:
#@title 4. Choose a filter, play with the parameters { display-mode: "form" }
filter = 'Sobel' #@param ["Mean", "Gaussian", "Sobel"]

if filter == 'Mean':
  Kernel_value = -20 #@param {type:"slider", min:-50, max:50, step:5}
  Kernel_size = 1 #@param {type:"slider", min:1, max:10, step:1}
  img_gray =  skimage.color.rgb2gray(img)
  my_img_float=skimage.util.img_as_float(img_gray) 
  mean_kernel = np.full((Kernel_size,Kernel_size),Kernel_value)
  mean_filter = ndi.correlate(my_img_float, mean_kernel)
  plt.imshow(mean_filter , cmap = 'Greys')
  io.imsave(f'{folder_path}/mean_filter.jpg',mean_filter)


if filter == 'Gaussian':
  sigma = 5 #@param {type:"slider", min:0, max:50, step:5}
  gaussian_using_skimage = skimage.filters.gaussian(img, sigma=sigma , mode='constant', cval=0.0)
  print(gaussian_using_skimage.shape)
  plt.imshow(gaussian_using_skimage)
  io.imsave(f'{folder_path}/Gaussian.alpha.png',gaussian_using_skimage)
  
if filter == 'Sobel':
  img_gray =  skimage.color.rgb2gray(img)
  my_img_float=skimage.util.img_as_float(img_gray) 
  sobel_filterred = filters.sobel(my_img_float)
  plt.imshow(sobel_filterred, cmap = 'gray')
  io.imsave(f'{folder_path}/Sobel filterred.jpg',sobel_filterred)



# download the image to the user, using files.download()
print('Do you want to download the photo?')

button1 = widgets.Button(description="Yes")
output1 = widgets.Output()

button2 = widgets.Button(description="NO")
output2 = widgets.Output()

def on_button_clicked(b):
  # Display the message within the output widget.
  if b == button1:
    with output1:
      print("Cool, now the image is on your computer!")
    if filter == 'Mean':
      files.download(f'{folder_path}/mean_filter.jpg')
    if filter == 'Gaussian':
      files.download(f'{folder_path}/Gaussian.alpha.png') #the gaussian is rgba
    if filter == 'Sobel':
      files.download(f'{folder_path}/Sobel filterred.jpg')

  else:
    with output1:
      print("It's O.K. you can keep play until you found the one you want")

button1.on_click(on_button_clicked)
display(button1, output1)

button2.on_click(on_button_clicked)
display(button2, output2)

In [ ]:
#@title 5. Image segmentation will display the segmented image with the object area, and download the segmented image to your computer. { display-mode: "form" }
segmentation_method = 'Otsu' #@param ["Otsu", "KMeans"]
k_for_kmeans = 3 #@param {type:"slider", min:2, max:10, step:1}

if segmentation_method == 'KMeans':
  attempts_for_kmeans = 9 #@param {type:"slider", min:1, max:10, step:1}
  object_index = 2 #@param {type:"slider", min:1, max:11, step:1}
  # Convert MxNx3 image into Kx3 where K=MxN
  pixel_values  = img.reshape((-1,3))  #-1 reshape means, in this case MxN
  #We convert the unit8 values to float as it is a requirement of the k-means method of OpenCV
  pixel_values = np.float32(pixel_values)
  criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
  _, labels, (centers) = cv2.kmeans(pixel_values, k_for_kmeans, None, criteria, attempts_for_kmeans, cv2.KMEANS_RANDOM_CENTERS)
  # flatten the labels array
  labels = labels.flatten()
  # convert all pixels to the color of the centroids
  segmented_image = centers[labels.flatten()]
# reshape back to the original image dimension
  segmented_image = segmented_image.reshape(img.shape)
# we take a look at the centers of the diff objects
# copy source img
  masked_image = img.copy()

# convert to the shape of a vector of pixel values (like suits for kmeans)
  masked_image = masked_image.reshape((-1, 3))

# color (i.e cluster) to exclude
  list_of_cluster_numbers_to_exclude = list(range(k_for_kmeans)) # create a list that has the number from 0 to k-1
  list_of_cluster_numbers_to_exclude.remove(object_index) # remove the cluster of grass that we want to keep, and not black out
  for cluster in list_of_cluster_numbers_to_exclude:
    masked_image[labels== cluster] = [0, 0, 0] # black all clusters except cluster 3

# convert back to original shape
  masked_image = masked_image.reshape(img.shape)
  plt.imshow(masked_image)
  io.imsave(f'{folder_path}/Kmeans object segmentations.jpg',masked_image)
  

  # detect contours in the mask and grab the largest one — this contour will represent the outline/boundary of a given object in the image.
  gray_mask=cv2.cvtColor((masked_image), cv2.COLOR_BGR2GRAY)
  cnts = cv2.findContours(gray_mask, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
  cnts = imutils.grab_contours(cnts)
  c = max(cnts, key=cv2.contourArea)
  total_area = []
  # calc area and append to list
  area = cv2.contourArea(c)
  total_area.append(area)
  
  plt.title(f'Your segmented image , The total area of the segmented object is={total_area}')
  
if segmentation_method == 'Otsu':
  # convert the image to grayscale
  gray_image = skimage.color.rgb2gray(img)
# blur the image to denoise
  blurred_image = skimage.filters.gaussian(gray_image, sigma=1.0)
# perform automatic thresholding
  t = skimage.filters.threshold_otsu(blurred_image)
  binary_mask = blurred_image > t
  selection = img.copy()
  selection[~binary_mask] = 0

  # detect contours in the mask and grab the largest one — this contour will represent the outline/boundary of a given object in the image.
  gray_mask=cv2.cvtColor((selection), cv2.COLOR_BGR2GRAY)
  cnts = cv2.findContours(gray_mask, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
  cnts = imutils.grab_contours(cnts)
  c = max(cnts, key=cv2.contourArea)
  total_area = []
  # calc area and append to list
  area = cv2.contourArea(c)
  total_area.append(area)
  
  plt.title(f'Your segmented image , The total area of the segmented object is={total_area}')
  
  plt.imshow(selection, cmap = 'Greys')
  io.imsave(f'{folder_path}/Otsu segmentation image.jpg',selection)
  




# download the image to the user, using files.download()
print('Do you want to download the photo?')

button1 = widgets.Button(description="Yes")
output1 = widgets.Output()

button2 = widgets.Button(description="NO")
output2 = widgets.Output()

def on_button_clicked(b):
  # Display the message within the output widget.
  if b == button1:
    with output1:
      print("Cool, now the image is on your computer!")
    if segmentation_method == 'KMeans':
      files.download(f'{folder_path}/Kmeans object segmentations.jpg')
    if segmentation_method == 'Otsu':
      files.download(f'{folder_path}/Otsu segmentation image.jpg')


  else:
    with output1:
      print("It's O.K. you can keep play until you found the one you want")

button1.on_click(on_button_clicked)
display(button1, output1)

button2.on_click(on_button_clicked)
display(button2, output2)